# Mosaic Monthly

_by Alex Berndt_
_12 Dec 2021_

__Abstract__

This notebook serves as an entrypoint into what a Spatio-Temporal Asset Catalog (STAC) is, and how we use this at Overstory.

__References__

1. [Example STAC Catalog from Planet Labs](https://developers.planet.com/planetschool/introduction-to-stac-part-2-creating-an-example-stac-catalog-of-planet-imagery-with-pystac/)

In [3]:
import numpy as np
import xarray as xr

import rasterio.features
import stackstac
import pystac_client
import planetary_computer

import xrspatial.multispectral as ms

from dask_gateway import GatewayCluster

/home/alex/github_repos/ml-notebooks/.venv/lib/python3.8/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes


In [4]:
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [-122.27508544921875, 47.54687159892238],
            [-121.96128845214844, 47.54687159892238],
            [-121.96128845214844, 47.745787772920934],
            [-122.27508544921875, 47.745787772920934],
            [-122.27508544921875, 47.54687159892238],
        ]
    ],
}
bbox = rasterio.features.bounds(area_of_interest)

In [11]:
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = stac.search(
    bbox=bbox,
    datetime="2020-11-01/2020-12-31",
    collections=["sentinel-2-l2a"],
    limit=500,  # fetch items in batches of 500
    query={"eo:cloud_cover": {"lt": 25}},
)

items = list(search.get_items())
print(len(items))

5


In [12]:
signed_items = [planetary_computer.sign(item).to_dict() for item in items]

In [13]:
data = (
    stackstac.stack(
        signed_items,
        assets=["B04", "B03", "B02"],  # red, green, blue
        chunksize=4096,
        resolution=100,
    )
    .where(lambda x: x > 0, other=np.nan)  # sentinel-2 uses 0 as nodata
    .assign_coords(band=lambda x: x.common_name.rename("band"))  # use common names
)
data

<xarray.DataArray 'stackstac-4b19852470c9b2594f369aaf63d5c24d' (time: 5, band: 3, y: 1099, x: 1099)>
dask.array<where, shape=(5, 3, 1099, 1099), dtype=float64, chunksize=(1, 1, 1099, 1099), chunktype=numpy.ndarray>
Coordinates:
  * time                                     (time) datetime64[ns] 2020-11-01...
    id                                       (time) <U54 'S2B_MSIL2A_20201101...
  * band                                     (band) <U5 'red' 'green' 'blue'
  * x                                        (x) float64 4.999e+05 ... 6.097e+05
  * y                                        (y) float64 5.3e+06 ... 5.19e+06
    sat:orbit_state                          <U10 'descending'
    s2:datastrip_id                          (time) <U64 'S2B_OPER_MSI_L2A_DS...
    s2:product_uri                           (time) <U65 'S2B_MSIL2A_20201101...
    sat:relative_orbit                       (time) int64 56 13 56 56 13
    s2:mean_solar_zenith                     (time) float64 62.51 64.9 ... 71.85
    eo:cloud_cover                           (time) float64 0.2855 ... 22.94
    s2:mean_solar_azimuth                    (time) float64 171.5 ... 165.6
    s2:thin_cirrus_percentage                (time) float64 0.000184 ... 0.1651
    s2:saturated_defective_pixel_percentage  float64 0.0
    s2:processing_baseline                   <U5 '02.12'
    s2:mgrs_tile                             <U5 '10TET'
    s2:reflectance_conversion_factor         (time) float64 1.014 ... 1.034
    s2:dark_features_percentage              (time) float64 6.867 ... 14.56
    s2:product_type                          <U7 'S2MSI2A'
    s2:nodata_pixel_percentage               (time) float64 1.074 ... 3.947
    s2:unclassified_percentage               (time) float64 3.068 4.378 ... 8.58
    constellation                            <U10 'Sentinel 2'
    s2:not_vegetated_percentage              (time) float64 4.861 ... 1.764
    s2:degraded_msi_data_percentage          float64 0.0
    s2:datatake_id                           (time) <U34 'GS2B_20201101T19153...
    s2:generation_time                       (time) <U24 '2020-11-02T21:15:03...
    proj:epsg                                int64 32610
    s2:granule_id                            (time) <U62 'S2B_OPER_MSI_L2A_TL...
    s2:datatake_type                         <U8 'INS-NOBS'
    s2:water_percentage                      (time) float64 12.89 ... 9.798
    s2:snow_ice_percentage                   (time) float64 0.164 ... 1.404
    platform                                 <U11 'Sentinel-2B'
    s2:vegetation_percentage                 (time) float64 70.82 ... 32.16
    s2:high_proba_clouds_percentage          (time) float64 0.2196 ... 16.31
    instruments                              <U3 'msi'
    s2:medium_proba_clouds_percentage        (time) float64 0.06571 ... 6.466
    s2:cloud_shadow_percentage               (time) float64 1.041 ... 8.791
    gsd                                      int64 10
    title                                    (band) <U20 'Band 4 - Red - 10m'...
    proj:bbox                                object {5190240.0, 609780.0, 499...
    proj:shape                               object {10980}
    proj:transform                           object {0.0, 5300040.0, 10.0, 49...
    common_name                              (band) <U5 'red' 'green' 'blue'
    center_wavelength                        (band) float64 0.665 0.56 0.49
    full_width_half_max                      (band) float64 0.038 0.045 0.098
    epsg                                     int64 32610
Attributes:
    spec:        RasterSpec(epsg=32610, bounds=(499900, 5190200, 609800, 5300...
    crs:         epsg:32610
    transform:   | 100.00, 0.00, 499900.00|\n| 0.00,-100.00, 5300100.00|\n| 0...
    resolution:  100

In [15]:
# data = data.persist()

In [ ]:
median = data.median(dim="time").compute()

/home/alex/github_repos/ml-notebooks/.venv/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


In [ ]:
image = ms.true_color(*median)  # expects red, green, blue DataArrays

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 8))

ax.set_axis_off()
image.plot.imshow(ax=ax);

In [ ]:
monthly = data.groupby("time.month").median().compute()

In [ ]:
images = [ms.true_color(*x) for x in monthly]
images = xr.concat(images, dim="time")

g = images.plot.imshow(x="x", y="y", rgb="band", col="time", col_wrap=3, figsize=(6, 8))
for ax in g.axes.flat:
    ax.set_axis_off()

plt.tight_layout()